<a href="https://colab.research.google.com/github/stefanoridolfi/Coding_ML/blob/master/MLP_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:



commento="VERSIONE con BIAS"
import matplotlib.pyplot as plt
import random
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

import pandas as pd
from csv import reader
from math import sqrt,exp
import matplotlib.pyplot as plt




@np.vectorize

def posizione(classe):
  
  if classe==1:
    formato = 'ro'
  else:
    formato = 'go'
  return formato

def posizione_neurale(p):
    if p<0.5:
      formato = 'ro'
      out=1
    else:
      formato = 'go'
      out=0
    return out,formato



def sigmoid(x):
    return 1 / (1 + np.e ** -x)
activation_function = sigmoid

def der_sig(x):
  return sigmoid(x)*(1-sigmoid(x))

def float_column_to_int(dataset, column):
  for row in dataset:
    row[column] = int(row[column])

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
  correct = 0
  for i in range(len(actual)):
    if actual[i] == predicted[i]:
      correct += 1
  return correct / float(len(actual)) * 100.0

########################

###########INPUT##################
learning_rate=0.01
hid1=10
hid2=10
epoche=5
num_rows=8000
####################
# dataset mio quadrato ##############
CSV_url='https://raw.githubusercontent.com/stefanoridolfi/Coding_ML/master/dataset/dataset_quadrato.csv'
#############
df=pd.read_csv(CSV_url,nrows=num_rows)
df = df.values
np.random.seed(5)
np.random.shuffle(df)
train = df[0:int(0.7*len(df))]
test = df[int(0.7*len(df)):int(len(df))]

xy_train = train[:, 0:3]
xy_test = test[:, 0:3]

xy_train_list=xy_train.tolist()
xy_test_list = xy_test.tolist()
float_column_to_int(xy_train_list, 2)
float_column_to_int(xy_test_list, 2)



'''
Creo matrice dei pesi. Input sono 3, x1,x2 e x3 che è BIAS. Il vettore X ha dimensioni 1x3. Lo strato hidden è composto da 4 uscite, y1,y2,y3 e y4 ed è rappresentato dal vettore Y di dimensioni 4x1
Il vettore Y, di dimensioni 4x1 è il risultato del prodotto Wih.dot.X La matrice Wih ha dimensioni 4x3. Y=Wih.dot.X (4x1=4x3.dot.3x1)
L'output Z ha dimensioni 1. La matrice dei pesi da Y a Z,matrice Who,  ha dimensione 1x4. L'output Z ha dimensioni 1x1. Per questo Z=Who.dot.Y (1x1=1x4.dot.4x1)  
'''
#Creo punti Input per training. Per ogni punto aggiungo un campo che vale 1 se il punto è nel quadrato, 0 se è fuori. Nel disegno 1 è punto rosso 0 è punto verde
# defnisco contorni del quadrato


# Creo la matrice W1 , hid1,n: n sono gl input  i cui elementi sono i pesi. I pesi hanno un valore con distrubuzione gaussiano vm==0 e varianza=1
W1=np.random.randn(hid1,2)

# Creo la matrice W2, hid2,hid1, i cui elementi sono i pesi. I pesi hanno un valore distrubuzione gaussiano vm==0 e varianza=1
W2=np.random.randn(hid2,hid1)

# Creo la matrice W2, m,hid2: m sono gli output i cui elementi sono i pesi. I pesi hanno un valore distrubuzione gaussiano vm==0 e varianza=1
W3=np.random.randn(1,hid2)

# Creo il vettore BIAS del primo livello nascosto il vettore ha dimensioni (hid1x1)
b1=[]
for t in range(0,hid1):
  b1.append(1)
b1=np.array(b1)
b1=b1.reshape(hid1,1)
# Creo il vettore BIAS del secondo livello nascosto il vettore ha dimensioni (hid2x1)
b2=[]
for t in range(0,hid2):
  b2.append(1)
b2=np.array(b2)
b2=b2.reshape(hid2,1)
#Creo il vettore BIAS del livello di uscita, il vettore ha dimensioni (m x 1)
b3=[]
for t in range(0,1):
  b3.append(1)
b3=np.array(b3)
b3=b3.reshape(1,1)
###############TRAINING################################ 
print("INIZIO TRAINING SET\n")

Epo=[]
Err_y=[]

for e in range(0,epoche):
  Err=0
  Err_rel=0
  for punto in xy_train_list:
    X=(punto[0],punto[1])
# lo devo trasformare in un array altrimenti non posso fare operazioni vettoriali
    X=np.array(X)
    X=X.reshape(2,1)
  

 
#calcolo Y=W1.dot.X
    Y=np.dot(W1,X)
#Sommo ad Y il BIAS b1
    Y=Y+b1
 #calcolo Y_act che è il vettore Y a cui ho applicato la funzione di attivazione
    Y_act=activation_function(Y)

#Calcolo S=W2.dot.Y.T
    S=np.dot(W2,Y_act)
#Sommo ad S il BIAS b2
    S=S+b2
 #calcolo S_act che è il vettore S a cui ho applicato la funzione di attivazione
    S_act=activation_function(S)

#Calcolo Z=W3.dot.S.T
    Z=np.dot(W3,S_act)
#Sommo ad Z il BIAS b2
    Z=Z+b3
#calcolo Z_act che è il vettore Z a cui ho applicato la funzione di attivazione
    Z_act=activation_function(Z)


    error_output=punto[2]-Z_act
    Err=Err+(error_output)**2

# Calcolo i delta ed i d_W
    delta3=error_output*der_sig(Z)
    d_W3=np.dot(delta3,Z_act.T)
    delta2=np.dot(W3.T,delta3)*der_sig(S)
    d_W2=np.dot(delta2,Y_act.T)
  
    #delta1=np.dot(W2.T,delta2)*der_sig(Y)
    d_W1=np.dot(delta1,X.T)
    
    W1=W1+learning_rate*d_W1
    b1=b1+learning_rate*b1
    W2=W2+learning_rate*d_W2
    b2=b2+learning_rate*b2
    W3=W3+learning_rate*d_W3
    b3=b3+learning_rate*b3
  Err_rel=(Err)/len(xy_train_list)
  Epo.append(e)
  Err_y.append(Err_rel)
  print("epoca=",e,"Errore =",Err_rel)
  

######################FINE TRAINING#######################################
# TEST SET  
print("INIZIO TEST\n")
predicted=list()
errori_tot=0
for punto in xy_test_list:
   
   X=(punto[0],punto[1])
   X=np.array(X)
   X=X.reshape(2,1)
   Y=np.dot(W1,X)+b1
   Y_act=activation_function(Y)
   S=np.dot(W2,Y_act)+b2
   S_act=activation_function(S)
   Z=np.dot(W3,S_act)+b3
   Z_act=activation_function(Z)
# inserisco valore predetto
   predicted.append(posizione_neurale(Z_act)[0])
   if posizione_neurale(Z_act)[0]!=punto[2]:
     #pint("Z_act:formato neurale, output neurale, posizione corretta",Z_act,posizione_neurale(Z_act)[1],",",posizione_neurale(Z_act)[0],posizione(punto,xa,xb,ya,yb)[0])
     errori_tot=errori_tot+1
actual = [row[-1] for row in xy_test_list]
accuracy=accuracy_metric(actual, predicted)
print(commento," punti training=",len(xy_train_list)," epoche=",epoche," punti test=",len(xy_test_list), " learning_rate=",learning_rate, " neuroni strati nascosti=",hid1,hid2)
print("errori totali=",errori_tot, "su", len(xy_test_list), "%",(errori_tot/len(xy_test_list))*100)
print("accuracy native python: %2.2f" % accuracy)

#################################USING SCIKIT###################
from sklearn.neural_network import MLPClassifier
CSV_url='https://raw.githubusercontent.com/stefanoridolfi/Coding_ML/master/dataset/dataset_quadrato_with_bias.csv'
df1=pd.read_csv(CSV_url,nrows=num_rows)
df1=df1.values
np.random.seed(5)
np.random.shuffle(df1)


train1 = df1[0:int(0.7*len(df1))]
test1 = df1[int(0.7*len(df1)):int(len(df1))]



x_train1 = train1[:,0:3] 
y_train1 = train1[:, 3] 

x_test1 = test1[:, 0:3]
y_test1 = test1[:, 3]
clf = MLPClassifier(solver='lbfgs', activation='relu',alpha=1e-5,hidden_layer_sizes=(10), random_state=1)
clf.fit(x_train1, y_train1)
predicted1=clf.predict(x_test1)
accuracy1=accuracy_metric(y_test1, predicted1)
print("accuracy scikit: %2.2f" % accuracy1)



INIZIO TRAINING SET

epoca= 0 Errore = [[0.93546188]]
epoca= 1 Errore = [[0.96232143]]
epoca= 2 Errore = [[0.96232143]]
epoca= 3 Errore = [[0.96232143]]
epoca= 4 Errore = [[0.96232143]]
INIZIO TEST

VERSIONE con BIAS  punti training= 5600  epoche= 5  punti test= 2400  learning_rate= 0.01  neuroni strati nascosti= 10 10
errori totali= 78 su 2400 % 3.25
accuracy native python: 96.75
accuracy scikit: 96.75
